# Deadbeat controller for the double integrator

Consider a mass of 1kg moving in one direction on a friction-free horizontal surface. We can apply a force to the mass (input signal $u$), and the mass is also subject to disturbance forces $v$. We are interested in controlling the position $z$ of the mass. In continuous time the dynamics are described by
$$ \ddot{z} = u + v, $$
which can also be represented as a state-space system with the state vector $x = \begin{bmatrix} z & \dot{z}\end{bmatrix}^T$ as 
\begin{align}
\dot{x} &= \underbrace{\begin{bmatrix} 0 & 1\\0 & 0\end{bmatrix}}_{A}x + \underbrace{\begin{bmatrix}0\\1\end{bmatrix}}_{B}u + \underbrace{\begin{bmatrix}0\\1\end{bmatrix}}_{B}v\\
y &= \underbrace{\begin{bmatrix}1 & 0 \end{bmatrix}}_C x
\end{align}

## Discrete-time state-space model
The discrete-time model for a sampling period $h$ is
\begin{align}
x(k+1) &= \Phi(h)x(k) + \Gamma(h)u + \Gamma(h)v\\
y(k) &= Cx(k)
\end{align}
where
$$ \Phi(h) = \mathrm{e}^{Ah} = \begin{bmatrix} 1 & h\\0 & 1 \end{bmatrix}$$
and
$$ \Gamma(h) = \int_0^h \mathrm{e}^{As}B ds = \begin{bmatrix} \frac{h^2}{2}\\h \end{bmatrix}.$$

In [6]:
# Verify
import numpy as np
import sympy as sy
sy.init_printing(use_latex='mathjax')

h = sy.symbols('h', real=True, positive=True)
A = sy.Matrix([[0,1], [0,0]])
B = sy.Matrix([[0],[1]])
Phi = sy.simplify(sy.exp(A*h))
Phi

⎡1  h⎤
⎢    ⎥
⎣0  1⎦

In [9]:
s = sy.symbols('s')
Gamma = sy.integrate(sy.exp(A*s)*B, (s, 0, h))
Gamma

⎡ 2⎤
⎢h ⎥
⎢──⎥
⎢2 ⎥
⎢  ⎥
⎣h ⎦

## Reachability 
The controllability matrix for this second order system becomes
$$ W_c = \begin{bmatrix} \Gamma & \Phi\Gamma \end{bmatrix} = \begin{bmatrix} \frac{h^2}{2} & \frac{3h^2}{2}\\h & h\end{bmatrix}, $$
with determinant
$$\det W_c = \frac{h^3}{2}(1 - 3) = -h^3,$$
which is different from zero since $h>0$.
It is hence possible to reach any point in the state-space from any other point in just two steps (two sampling periods).

In [16]:
# Verify
Wc = sy.BlockMatrix([[Gamma, Phi*Gamma]]).as_explicit()
Wc

⎡ 2     2⎤
⎢h   3⋅h ⎥
⎢──  ────⎥
⎢2    2  ⎥
⎢        ⎥
⎣h    h  ⎦

In [17]:
sy.det(Wc)

  3
-h 

### Designing an input sequence
We now know that the system is reachable. This means that we can take the system from the origin in the state-space (position zero and velocity zero) to any other point in state-space. And it can be done in only two steps with the input sequence
$$ u(0), \, u(1).$$
Let's say we want to reach the point
$$ x_d = \begin{bmatrix} a\\b \end{bmatrix},$$
which in words is that we want the mass to be at $z=a$ and having the velocity $\dot{z}=b$. The general solution for an n-th order discrete-time state space system is
\begin{align}
x(n) &= \Phi^n x(0) + \Phi^{n-1}\Gamma u(0) + \Phi^{n-2}\Gamma u(1) + \cdots + \Gamma u(n-1)\\
&= \Phi^n x(0) + W_cU, 
\end{align}
where
$$ U = \begin{bmatrix} u(n-1)\\u(n-2)\\\vdots\\u(0)\end{bmatrix}. $$
In the case here we have $x(0)=0$ and this leads to the equation
$$ W_cU = x_d, \qquad \text{with solution}$$
\begin{align}
U &= \begin{bmatrix}u(1)\\u(0)\end{bmatrix} = W_c^{-1}x_d = \begin{bmatrix} \frac{h^2}{2} & \frac{3h^2}{2}\\h & h\end{bmatrix}^{-1} \begin{bmatrix} a\\b \end{bmatrix}\\
&= \frac{1}{-h^3} \begin{bmatrix} h & -\frac{3h^2}{2}\\-h & \frac{h^2}{2} \end{bmatrix} \begin{bmatrix} a\\b \end{bmatrix}\\
&= \begin{bmatrix} -\frac{1}{h^2} & \frac{3}{2h}\\\frac{1}{h^2} & -\frac{1}{2h} \end{bmatrix} \begin{bmatrix} a\\b \end{bmatrix}\\
&= \begin{bmatrix} -\frac{a}{h^2} + \frac{3b}{2h}\\ \frac{a}{h^2} - \frac{b}{2h} \end{bmatrix}.
\end{align}
Thus the input sequence becomes $u(0) = \frac{a}{h^2} - \frac{b}{2h}$, $u(1) = \frac{-a}{h^2} + \frac{3b}{2h}$.

In [19]:
# Verify
a,b = sy.symbols('a,b')
U = Wc.inv()*sy.Matrix([[a],[b]])
U

⎡  a    3⋅b⎤
⎢- ── + ───⎥
⎢   2   2⋅h⎥
⎢  h       ⎥
⎢          ⎥
⎢ a     b  ⎥
⎢ ── - ─── ⎥
⎢  2   2⋅h ⎥
⎣ h        ⎦

In [21]:
# Simulate
u0 = U[1,0]
u1 = U[0,0]
x0 = sy.Matrix([[0],[0]])
x1 = Phi*x0 + Gamma*u0
x2 = Phi*x1 + Gamma*u1
sy.simplify(x2)

⎡a⎤
⎢ ⎥
⎣b⎦

## State feedback
Introducing the state-feedback control law
$$ u = -l_1x_1 - l_2 x_2 + l_0y_{ref} = -Lx + l_0y_{ref}$$
gives the closed-loop state-space system
\begin{align}
x(k+1) &= \Phi x(k) +\Gamma\big(-Lx(k) + l_0y_{ref}(k)\big) + \Gamma v(k) = \left( \Phi - \Gamma L \right) x(k) + l_0\Gamma y_{ref}(k) + \Gamma v(k)\\
y(k) &= C x(k)
\end{align}
with characteristic polynomial given by
\begin{align}
\det \left( zI - (\Phi-\Gamma L) \right) &= \det \left( \begin{bmatrix} z & 0\\0 & z \end{bmatrix} - \begin{bmatrix} 1 & h\\0 & 1 \end{bmatrix} + \begin{bmatrix} l_1\frac{h^2}{2} & l_2\frac{h^2}{2}\\ l_1h & l_2h \end{bmatrix} \right)\\
&= \det \begin{bmatrix} z-1+l_1\frac{h^2}{2} & -h+l_2\frac{h^2}{2}\\l_1h & z-1+l_2h 
\end{bmatrix}\\
&= (z-1+l_1\frac{h^2}{2})(z-1+l_2h) - l_1h(-h + l_2\frac{h^2}{2})\\
\end{align}

\begin{align}
&= z^2 + (-1+l_2h-1+l_1\frac{h^2}{2}) z + (l1\frac{h^2}{2}l_2h +l_1h^2 -l_1l_2\frac{h^2}{2})\\
&= z^2 + (l_1\frac{h^2}{2}+l_2h-2) z + l_1h^2
\end{align}


In [27]:
# Verify
l1, l2 = sy.symbols('l1, l2', real=True)
z = sy.symbols('z')
L = sy.Matrix([[l1, l2]])
ch_poly = sy.Poly((z*sy.eye(2) - (Phi - Gamma*L)).det(), z)
ch_poly.as_expr()

 2                    ⎛ 2              ⎞    
h ⋅l₁           2     ⎜h ⋅l₁           ⎟    
───── - h⋅l₂ + z  + z⋅⎜───── + h⋅l₂ - 2⎟ + 1
  2                   ⎝  2             ⎠    

In [23]:
sy.eye(3)

⎡1  0  0⎤
⎢       ⎥
⎢0  1  0⎥
⎢       ⎥
⎣0  0  1⎦